In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf

from sklearn.metrics import r2_score
from statsmodels.othermod.betareg import BetaModel

In [2]:
model_data = pd.read_parquet("data/model_data.parquet")
model_data = model_data[model_data['political_alignment'] != 'autre']
model_data["election_year"] = model_data["last_pres"].dt.year
model_data["decade"] = (model_data["month"].dt.year // 10) * 10
model_data['europ_votes_share_dum'] = (model_data['europ_votes_share'] * model_data['europ_dummy_short']).fillna(0)

In [3]:
nul_outcomes = (model_data["quotes_share"] == 0).sum()
print(nul_outcomes)
print(100 * nul_outcomes / len(model_data))

2385
19.371345029239766


In [4]:
unit_outcomes = (model_data["quotes_share"] == 1).sum()
print(unit_outcomes)

0


In [5]:
model_data_njl = pd.read_parquet("data/model_data_njl.parquet")
model_data_njl = model_data_njl[model_data_njl['political_alignment'] != 'autre']
model_data_njl["election_year"] = model_data_njl["last_pres"].dt.year
model_data_njl["decade"] = (model_data_njl["month"].dt.year // 10) * 10
model_data_njl['europ_votes_share_dum'] = (model_data_njl['europ_votes_share'] * model_data_njl['europ_dummy_short']).fillna(0)

In [6]:
nul_outcomes = (model_data_njl["quotes_share"] == 0).sum()
print(nul_outcomes)
print(nul_outcomes / len(model_data_njl))

409
0.12765293383270912


In [7]:
unit_outcomes = (model_data_njl["quotes_share"] == 1).sum()
print(unit_outcomes)

0


# Tentative de modélisation directe du nombre de citations

In [8]:
outcome = "quotes_nb"

In [9]:
alignment_ref = 'Far right'
period_ref = '1980'
journal_ref = 'Le Monde'

formula = f"{outcome} ~ C(decade, Treatment(reference={period_ref})) + C(political_alignment, Treatment(reference='{alignment_ref}')) + C(journal, Treatment(reference='{journal_ref}'))+ na_share + pres_votes_share + government"

model = smf.ols(formula, data=model_data).fit(cov_type='HC3')
y = model_data[outcome]
y_pred = model.predict(model_data)

r2 = r2_score(y, y_pred)
print(f"R2: {100*r2:.2f}%")
print("")

params = model.params.rename("coef").to_frame()
pvalues = model.pvalues.rename("pval").to_frame()
results = pd.merge(params, pvalues, left_index=True, right_index=True)
results.style

R2: 48.14%



,coef,pval
Intercept,845.430448,0.000000
"C(decade, Treatment(reference=1980))[T.1990]",562.426750,0.000000
"C(decade, Treatment(reference=1980))[T.2000]",748.883661,0.000000
"C(decade, Treatment(reference=1980))[T.2010]",977.715159,0.000000
"C(decade, Treatment(reference=1980))[T.2020]",1972.178088,0.000000
"C(political_alignment, Treatment(reference='Far right'))[T.Center]",-349.160998,0.000000
"C(political_alignment, Treatment(reference='Far right'))[T.Far left]",6.922613,0.765673
"C(political_alignment, Treatment(reference='Far right'))[T.Left]",-73.473529,0.018501
"C(political_alignment, Treatment(reference='Far right'))[T.Other]",-962.642153,0.000000
"C(political_alignment, Treatment(reference='Far right'))[T.Right]",-138.880159,0.000411


In [10]:
alignment_ref = 'Right'
period_ref = '1980'

formula = f"{outcome} ~ C(decade, Treatment(reference={period_ref})) * C(political_alignment, Treatment(reference='{alignment_ref}')) + na_share + journal + pres_votes_share + government"
model = smf.ols(formula, data=model_data).fit(cov_type='HC3')
y = model_data[outcome]
y_pred = model.predict(model_data)

r2 = r2_score(y, y_pred)
print(f"R2: {100*r2:.2f}%")
print("")

params = model.params.rename("coef").to_frame()
pvalues = model.pvalues.rename("pval").to_frame()
results = pd.merge(params, pvalues, left_index=True, right_index=True)
results.style

R2: 55.90%



,coef,pval
Intercept,-543.376622,0.000000
"C(decade, Treatment(reference=1980))[T.1990]",510.559027,0.000000
"C(decade, Treatment(reference=1980))[T.2000]",873.644033,0.000000
"C(decade, Treatment(reference=1980))[T.2010]",1122.955470,0.000000
"C(decade, Treatment(reference=1980))[T.2020]",2343.572656,0.000000
"C(political_alignment, Treatment(reference='Right'))[T.Center]",-208.918424,0.001388
"C(political_alignment, Treatment(reference='Right'))[T.Far left]",95.241376,0.046125
"C(political_alignment, Treatment(reference='Right'))[T.Far right]",110.535191,0.061176
"C(political_alignment, Treatment(reference='Right'))[T.Left]",10.417476,0.784897
"C(political_alignment, Treatment(reference='Right'))[T.Other]",-266.753500,0.000027


# Modèle à probabilité linéaire

In [11]:
outcome = "quotes_share"
model_data = model_data_njl

In [12]:
formula = f"{outcome} ~ na_share + pres_votes_share + government"

model = smf.ols(formula, data=model_data).fit(cov_type='HC3')
y = model_data[outcome]
y_pred = model.predict(model_data)

r2 = r2_score(y, y_pred)
print(f"R2: {100*r2:.2f}%")
print("")

params = model.params.rename("coef").to_frame()
pvalues = model.pvalues.rename("pval").to_frame()
results = pd.merge(params, pvalues, left_index=True, right_index=True)
results.style.format({"coef": "{:.4f}", "pval": "{:.4f}"})

R2: 89.18%



,coef,pval
Intercept,-0.0115,0.0000
na_share,0.6544,0.0000
pres_votes_share,0.3214,0.0000
government,0.0934,0.0000


In [13]:
print((y_pred < 0).sum())
print((y_pred > 1).sum())
print(len(y_pred))
print(100 * (y_pred < 0).sum() / len(y_pred))

776
0
3204
24.21972534332085


In [14]:
outcome_data = pd.DataFrame({
    "month": model_data["month"],
    "y_pred": y_pred
}).groupby(['month']).sum()
bad_outcomes = (outcome_data > 1).sum().iloc[0]
print(bad_outcomes)
print(100 * bad_outcomes / len(y_pred))

412
12.85892634207241


# Modèle logistique

In [15]:
formula = f"{outcome} ~ na_share + pres_votes_share + government"
model = smf.logit(formula, data=model_data).fit(cov_type='HC3')

params = model.params.rename("coef").to_frame()
pvalues = model.pvalues.rename("pval").to_frame()
odds_ratios = np.exp(model.params).rename("odds_ratio").to_frame()
results = params.join(odds_ratios).join(pvalues)
results.style.format({"coef": "{:.4f}", "odds_ratio": "{:.4f}", "pval": "{:.4f}"})

Optimization terminated successfully.
         Current function value: 0.226831
         Iterations 7


,coef,odds_ratio,pval
Intercept,-3.7914,0.0226,0.0000
na_share,4.0559,57.7382,0.0000
pres_votes_share,4.4961,89.6655,0.0000
government,0.2638,1.3019,0.0000


In [16]:
y_pred = model.predict(model_data)

outcome_data = pd.DataFrame({
    "month": model_data["month"],
    "y_pred": y_pred
}).groupby(['month']).sum()

bad_outcomes = (outcome_data > 1).sum().iloc[0]
print(bad_outcomes)
print(100 * bad_outcomes / len(y_pred))

318
9.925093632958802


# Régression béta

In [17]:
# Make sure the outcome is strictly in (0, 1)
eps = 1e-6
model_data[outcome] = model_data[outcome].clip(eps, 1 - eps)

formula = f"{outcome} ~ na_share + pres_votes_share + government"
mod = BetaModel.from_formula(formula, data=model_data)
res = mod.fit()
print(res.summary())
print("")

params = res.params.rename("coef").to_frame()
pvalues = res.pvalues.rename("pval").to_frame()
exp_coef = np.exp(res.params).rename("exp_coef").to_frame()
results = params.join(exp_coef).join(pvalues)
results = results.round(4)
results

                              BetaModel Results                               
Dep. Variable:           quotes_share   Log-Likelihood:                 8688.8
Model:                      BetaModel   AIC:                        -1.737e+04
Method:            Maximum Likelihood   BIC:                        -1.734e+04
Date:                Mon, 29 Sep 2025                                         
Time:                        22:36:10                                         
No. Observations:                3204                                         
Df Residuals:                    3199                                         
Df Model:                           3                                         
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept           -4.0645      0.029   -138.617      0.000      -4.122      -4.007
na_share             3.4980      0

,coef,exp_coef,pval
Intercept,-4.0645,0.0172,0.0
na_share,3.4980,33.0499,0.0
pres_votes_share,6.0247,413.5172,0.0
government,0.3229,1.3811,0.0
precision,2.5835,13.2429,0.0


In [18]:
y_pred = res.predict(model_data)

outcome_data = pd.DataFrame({
    "month": model_data["month"],
    "y_pred": y_pred
}).groupby(['month']).sum()

bad_outcomes = (outcome_data > 1).sum().iloc[0]
print(bad_outcomes)
print(100 * bad_outcomes / len(y_pred))

318
9.925093632958802
